In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
data = pd.read_csv('../input/tmnist-typeface-mnist/TMNIST_Data.csv')
data

In [ ]:
print(data['names'].unique())
print(data['names'].nunique())

In [ ]:
data['names'].value_counts()

In [ ]:
N=list(range(len(data)))
n=len(data)
print(n)
random.seed(2021)
random.shuffle(N)

In [ ]:
trainY=data.loc[N[0:(n//4)*3],'labels']
testY=data.loc[N[(n//4)*3:],'labels']
data1=data.drop(['names','labels'],axis=1)
trainX=data1.loc[N[0:(n//4)*3]]
testX=data1.loc[N[(n//4)*3:]]

In [ ]:
binencoder = LabelBinarizer()
y0 = trainY
y = binencoder.fit_transform(y0)
y

In [ ]:
X_images = trainX.values.reshape(-1,28,28)
test_images = testX.values.reshape(-1,28,28)

print(X_images.shape)
print(test_images.shape)

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(9,9))
for i in range(9):
    r=i//3
    c=i%3
    axs[r][c].set_xticks([])
    axs[r][c].set_yticks([])
    axs[r][c].imshow(X_images[i])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size = 0.2, random_state=90)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = (X_train/255).reshape(-1,28,28,1).astype('float32')
X_test = (X_test/255).reshape(-1,28,28,1).astype('float32')

In [ ]:
test_images = (test_images/255).reshape(-1,28,28,1).astype('float32')

In [ ]:
model = Sequential()
model.add(Conv2D(32,(4,4),input_shape = (28,28,1),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
result = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=92, verbose=2)
result

In [ ]:
his = pd.DataFrame(result.history)
his.loc[:, ['accuracy', 'val_accuracy']].plot()
his.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
y_pred = model.predict(test_images)
print(y_pred[0])

In [ ]:
y_pred2=[]
for item in y_pred:   
    y_pred2+=[np.argmax(item)   ]
PRED=np.array(y_pred2)
ANS=np.array(testY)
print(PRED[0:10])
print(ANS[0:10])

In [ ]:
accuracy_score(ANS,PRED)